### Import das dependencias

In [1]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### 1. Importar todos os datasets tratados

In [2]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

consumos_faturados_path = data_path + 'treated/consumos_faturados_por_municipio_10_anos.csv'
caracterizacao_cpe_path = data_path + 'treated/caracterizacao-pes-contrato-ativo.csv'
diagrama_carga_path = data_path + 'treated/diagrama_de_carga_por_mes.csv'

df_consumos_faturados = pd.read_csv(consumos_faturados_path)
df_caracterizacao_cpe = pd.read_csv(caracterizacao_cpe_path)
df_diagrama_carga = pd.read_csv(diagrama_carga_path)

### 2. Juntar os datasets

- 1. Primeiro juntar consumos faturados e caracterizacao cpe
    - A coleção de caracterizacao CPE tem uma categorização de nível de tensão diferente da coleção consumos faturados
        - Separando a "Baixa Tensão" em duas "Baixa Tensão Normal" e "Baixa Tensão Especial"
        - Temos então de tratar da coleção caracterizacao_cpe primeiro para tornar esses dois tipos num único tipo "Baixa Tensão"\
        - Criar colunas para cada tipo de instalação e depois somar os CPEs de acordo
    - Após esse tratamento, pode-se proceder à junção 
- 2. Segundo juntar com a coleção de diagramas de carga.
    - Criar colunas para Mobilidade Elétrica Sim ou Mobilidade Elétrica Não e depois somar os CPEs de acordo

Ver os valores possíveis de Nível de Tensão da coleção caracterizacao de cpe

In [3]:
df_caracterizacao_cpe['Nível de Tensão'].unique()

array(['Baixa Tensão Especial', 'Baixa Tensão Normal',
       'Muito Alta, Alta e Média Tensões'], dtype=object)

1. Tratamento coleção caracterizacao CPE

In [4]:
# Substituir valores de nível de tensão
df_caracterizacao_cpe['Nível de Tensão'] = df_caracterizacao_cpe['Nível de Tensão'].replace({'Baixa Tensão Normal': 'Baixa Tensão', 'Baixa Tensão Especial': 'Baixa Tensão'})

df_caracterizacao_cpe['Nível de Tensão'].unique()

array(['Baixa Tensão', 'Muito Alta, Alta e Média Tensões'], dtype=object)

2. Agrupar os valores de contagens de CPEs com a nova categoria "Baixa Tensão"

In [5]:
# Criar colunas para cada 'Tipo de Instalação' e somar 'CPEs (#)'
df_caracterizacao_cpe_grouped = df_caracterizacao_cpe.pivot_table(index=['Concelho', 'Nível de Tensão', 'Ano', 'Mês'], 
                                               columns='Tipo de Instalação', 
                                               values='CPEs (#)', 
                                               aggfunc='sum').reset_index()

# Preencher valores vazios com 0
df_caracterizacao_cpe_grouped.fillna(0, inplace=True)

# Renomear as colunas
df_caracterizacao_cpe_grouped.columns.name = None
df_caracterizacao_cpe_grouped.columns = [col if col in ['Concelho', 'Nível de Tensão', 'Ano', 'Mês'] else f"CPEs_{col}" for col in df_caracterizacao_cpe_grouped.columns]

df_caracterizacao_cpe_grouped.head()

,Concelho,Nível de Tensão,Ano,Mês,CPEs_Doméstico,CPEs_Iluminação Pública,CPEs_Não Doméstico,CPEs_Outros
0,Abrantes,Baixa Tensão,2022,3,20283.0,293.0,4334.0,83.0
1,Abrantes,Baixa Tensão,2022,4,20267.0,293.0,4337.0,83.0
2,Abrantes,Baixa Tensão,2022,5,20254.0,293.0,4324.0,82.0
3,Abrantes,Baixa Tensão,2022,6,20259.0,293.0,4327.0,82.0
4,Abrantes,Baixa Tensão,2022,7,20250.0,293.0,4320.0,83.0


3. Guardar ficheiro para verificar se está tudo certo

In [6]:
file_path = data_path + 'treated/caracterizacao-pes-contrato-ativo-agrupado.csv'

df_caracterizacao_cpe_grouped.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

DataFrame guardado em  ../data/treated/caracterizacao-pes-contrato-ativo-agrupado.csv


4. Junção das coleçoes consumos faturados e caracterizacao cpe

In [7]:
merged_df = pd.merge(df_consumos_faturados, df_caracterizacao_cpe_grouped, on=['Concelho', 'Ano', 'Mês', 'Nível de Tensão'], how='inner')

# Verificar nulls
null_counts = merged_df.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

merged_df.info()
merged_df.head(50)

Número de nulls por coluna:
Concelho                   0
Nível de Tensão            0
Ano                        0
Mês                        0
Energia Ativa (kWh)        0
CPEs_Doméstico             0
CPEs_Iluminação Pública    0
CPEs_Não Doméstico         0
CPEs_Outros                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12740 entries, 0 to 12739
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Concelho                 12740 non-null  object 
 1   Nível de Tensão          12740 non-null  object 
 2   Ano                      12740 non-null  int64  
 3   Mês                      12740 non-null  int64  
 4   Energia Ativa (kWh)      12740 non-null  float64
 5   CPEs_Doméstico           12740 non-null  float64
 6   CPEs_Iluminação Pública  12740 non-null  float64
 7   CPEs_Não Doméstico       12740 non-null  float64
 8   CPEs_Outros              12740 non-null  floa

,Concelho,Nível de Tensão,Ano,Mês,Energia Ativa (kWh),CPEs_Doméstico,CPEs_Iluminação Pública,CPEs_Não Doméstico,CPEs_Outros
0,Abrantes,Baixa Tensão,2022,3,6375824.308,20283.0,293.0,4334.0,83.0
1,Abrantes,Baixa Tensão,2022,4,5560992.246,20267.0,293.0,4337.0,83.0
2,Abrantes,Baixa Tensão,2022,5,5480833.643,20254.0,293.0,4324.0,82.0
3,Abrantes,Baixa Tensão,2022,6,5420579.537,20259.0,293.0,4327.0,82.0
4,Abrantes,Baixa Tensão,2022,7,6227622.409,20250.0,293.0,4320.0,83.0
5,Abrantes,Baixa Tensão,2022,8,5906036.429,20260.0,293.0,4318.0,83.0
6,Abrantes,Baixa Tensão,2022,9,5316878.736,20270.0,293.0,4311.0,81.0
7,Abrantes,Baixa Tensão,2022,10,5428732.778,20280.0,293.0,4302.0,81.0
8,Abrantes,Baixa Tensão,2022,11,5843266.356,20291.0,293.0,4308.0,80.0
9,Abrantes,Baixa Tensão,2022,12,6778130.292,20297.0,294.0,4312.0,82.0


5. No dataset diagramas de cargas, criar colunas para a mobilidade elétrica

In [8]:
# Criar colunas para 'Inclui Mobilidade Elétrica' e somar 'CPEs com DCs Recolhidos (#)'
df_pivoted_mobility = df_diagrama_carga.pivot_table(index=['Concelho', 'Ano', 'Mês'], 
                                                        columns='Inclui Mobilidade Elétrica', 
                                                        values='CPEs com DCs Recolhidos (#)', 
                                                        aggfunc='sum').reset_index()

# Valores NA preencher com 0
df_pivoted_mobility.fillna(0, inplace=True)

# Renomear as colunas
df_pivoted_mobility.columns.name = None 
df_pivoted_mobility.columns = [col if col in ['Concelho', 'Ano', 'Mês'] else f"CPEs_Mobilidade_Eletrica_{col}" for col in df_pivoted_mobility.columns]

df_pivoted_mobility.head()

,Concelho,Ano,Mês,CPEs_Mobilidade_Eletrica_Não,CPEs_Mobilidade_Eletrica_Sim
0,Abrantes,2022,8,11097.0,1.0
1,Abrantes,2022,9,11298.0,1.0
2,Abrantes,2022,10,11563.0,1.0
3,Abrantes,2022,11,11969.0,1.0
4,Abrantes,2022,12,12210.0,1.0


6. Juntar com a coleção diagramas de carga

In [9]:
merged_df = pd.merge(merged_df, df_pivoted_mobility, on=['Concelho', 'Ano', 'Mês'], how='inner')

# Verificar nulls
null_counts = merged_df.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

merged_df.info()
merged_df.head(50)

Número de nulls por coluna:
Concelho                        0
Nível de Tensão                 0
Ano                             0
Mês                             0
Energia Ativa (kWh)             0
CPEs_Doméstico                  0
CPEs_Iluminação Pública         0
CPEs_Não Doméstico              0
CPEs_Outros                     0
CPEs_Mobilidade_Eletrica_Não    0
CPEs_Mobilidade_Eletrica_Sim    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9790 entries, 0 to 9789
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Concelho                      9790 non-null   object 
 1   Nível de Tensão               9790 non-null   object 
 2   Ano                           9790 non-null   int64  
 3   Mês                           9790 non-null   int64  
 4   Energia Ativa (kWh)           9790 non-null   float64
 5   CPEs_Doméstico                9790 non-null   float64
 6   C

,Concelho,Nível de Tensão,Ano,Mês,Energia Ativa (kWh),CPEs_Doméstico,CPEs_Iluminação Pública,CPEs_Não Doméstico,CPEs_Outros,CPEs_Mobilidade_Eletrica_Não,CPEs_Mobilidade_Eletrica_Sim
0,Abrantes,Baixa Tensão,2022,8,5906036.429,20260.0,293.0,4318.0,83.0,11097.0,1.0
1,Abrantes,Baixa Tensão,2022,9,5316878.736,20270.0,293.0,4311.0,81.0,11298.0,1.0
2,Abrantes,Baixa Tensão,2022,10,5428732.778,20280.0,293.0,4302.0,81.0,11563.0,1.0
3,Abrantes,Baixa Tensão,2022,11,5843266.356,20291.0,293.0,4308.0,80.0,11969.0,1.0
4,Abrantes,Baixa Tensão,2022,12,6778130.292,20297.0,294.0,4312.0,82.0,12210.0,1.0
5,Abrantes,Baixa Tensão,2023,1,7369772.117,20298.0,294.0,4309.0,82.0,13334.0,1.0
6,Abrantes,Baixa Tensão,2023,2,6507806.283,20288.0,294.0,4310.0,113.0,13499.0,1.0
7,Abrantes,Baixa Tensão,2023,3,6077229.553,20280.0,294.0,4316.0,75.0,13973.0,1.0
8,Abrantes,Baixa Tensão,2023,4,5174260.623,20280.0,294.0,4316.0,75.0,14307.0,1.0
9,Abrantes,Baixa Tensão,2023,5,5241076.429,20280.0,294.0,4316.0,75.0,14194.0,1.0


7. Renomear as colunas para remover espaços e transformar tudo em letras minúsculas

In [10]:
# Renomear as colunas removendo espaços e convertendo para letras minúsculas
merged_df.columns = merged_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Exibir as novas colunas
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9790 entries, 0 to 9789
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   concelho                      9790 non-null   object 
 1   nível_de_tensão               9790 non-null   object 
 2   ano                           9790 non-null   int64  
 3   mês                           9790 non-null   int64  
 4   energia_ativa_(kwh)           9790 non-null   float64
 5   cpes_doméstico                9790 non-null   float64
 6   cpes_iluminação_pública       9790 non-null   float64
 7   cpes_não_doméstico            9790 non-null   float64
 8   cpes_outros                   9790 non-null   float64
 9   cpes_mobilidade_eletrica_não  9790 non-null   float64
 10  cpes_mobilidade_eletrica_sim  9790 non-null   float64
dtypes: float64(7), int64(2), object(2)
memory usage: 841.5+ KB


8. Colocar os tipos de tensão numa linha.
    - Criando duas colunas de energia ativa (um para baixa tensão e outro para alta)
    - Criando duas colunas para CPEs_domestico (um para baixa tensao e outro para alta)
    - Criando duas colunas para CPEs_iluminacao_publica (um para baixa tensao e outro para alta)
    - Criando duas colunas para CPEs_nao domestico (um para baixa tensao e outro para alta)
    - Criando duas colunas para CPEs_outros (um para baixa tensao e outro para alta)
    - Renomear as colunas de mobilidade eletrica para baixa tensao, porque estes só correspondem a CPEs de baixa tensão

In [11]:
merged_df['energia_ativa_(kwh)'] = merged_df['energia_ativa_(kwh)'].astype('int64')

# Criar novas colunas para energia ativa e CPES de baixa e alta tensão, inicialmente com valor zero
merged_df['energia_ativa_baixa_tensao_kwh'] = 0
merged_df['energia_ativa_alta_tensao_kwh'] = 0
merged_df['cpes_domestico_baixa_tensao'] = 0
merged_df['cpes_domestico_alta_tensao'] = 0
merged_df['cpes_iluminacao_publica_baixa_tensao'] = 0
merged_df['cpes_iluminacao_publica_alta_tensao'] = 0
merged_df['cpes_nao_domestico_baixa_tensao'] = 0
merged_df['cpes_nao_domestico_alta_tensao'] = 0
merged_df['cpes_outros_baixa_tensao'] = 0
merged_df['cpes_outros_alta_tensao'] = 0

# Filtrar os dados de alta tensão e baixa tensão
baixa_tensao_mask = merged_df['nível_de_tensão'].str.contains('Baixa Tensão')
alta_tensao_mask = merged_df['nível_de_tensão'].str.contains('Muito Alta, Alta e Média Tensões')

# Atribuir os valores correspondentes de energia ativa e CPES às colunas apropriadas de baixa e alta tensão
merged_df.loc[baixa_tensao_mask, 'energia_ativa_baixa_tensao_kwh'] = merged_df.loc[baixa_tensao_mask, 'energia_ativa_(kwh)']
merged_df.loc[alta_tensao_mask, 'energia_ativa_alta_tensao_kwh'] = merged_df.loc[alta_tensao_mask, 'energia_ativa_(kwh)']

merged_df.loc[baixa_tensao_mask, 'cpes_domestico_baixa_tensao'] = merged_df.loc[baixa_tensao_mask, 'cpes_doméstico']
merged_df.loc[alta_tensao_mask, 'cpes_domestico_alta_tensao'] = merged_df.loc[alta_tensao_mask, 'cpes_doméstico']

merged_df.loc[baixa_tensao_mask, 'cpes_iluminacao_publica_baixa_tensao'] = merged_df.loc[baixa_tensao_mask, 'cpes_iluminação_pública']
merged_df.loc[alta_tensao_mask, 'cpes_iluminacao_publica_alta_tensao'] = merged_df.loc[alta_tensao_mask, 'cpes_iluminação_pública']

merged_df.loc[baixa_tensao_mask, 'cpes_nao_domestico_baixa_tensao'] = merged_df.loc[baixa_tensao_mask, 'cpes_não_doméstico']
merged_df.loc[alta_tensao_mask, 'cpes_nao_domestico_alta_tensao'] = merged_df.loc[alta_tensao_mask, 'cpes_não_doméstico']

merged_df.loc[baixa_tensao_mask, 'cpes_outros_baixa_tensao'] = merged_df.loc[baixa_tensao_mask, 'cpes_outros']
merged_df.loc[alta_tensao_mask, 'cpes_outros_alta_tensao'] = merged_df.loc[alta_tensao_mask, 'cpes_outros']

merged_df['cpes_mobilidade_eletrica_nao_baixa_tensao'] = merged_df['cpes_mobilidade_eletrica_não']
merged_df['cpes_mobilidade_eletrica_sim_baixa_tensao'] = merged_df['cpes_mobilidade_eletrica_sim']

merged_df['cpes_mobilidade_eletrica_nao_baixa_tensao'] = merged_df['cpes_mobilidade_eletrica_nao_baixa_tensao'].astype('int64')
merged_df['cpes_mobilidade_eletrica_sim_baixa_tensao'] = merged_df['cpes_mobilidade_eletrica_sim_baixa_tensao'].astype('int64')

# Excluir a coluna de tipo de tensão original
merged_df.drop(columns=['nível_de_tensão'], inplace=True)

# Agrupar por ano e mês e somar os valores das colunas
merged_df = merged_df.groupby(['concelho', 'ano', 'mês']).sum().reset_index()

# Exibir os registros do DataFrame
merged_df


,concelho,ano,mês,energia_ativa_(kwh),cpes_doméstico,cpes_iluminação_pública,cpes_não_doméstico,cpes_outros,cpes_mobilidade_eletrica_não,cpes_mobilidade_eletrica_sim,...,cpes_domestico_baixa_tensao,cpes_domestico_alta_tensao,cpes_iluminacao_publica_baixa_tensao,cpes_iluminacao_publica_alta_tensao,cpes_nao_domestico_baixa_tensao,cpes_nao_domestico_alta_tensao,cpes_outros_baixa_tensao,cpes_outros_alta_tensao,cpes_mobilidade_eletrica_nao_baixa_tensao,cpes_mobilidade_eletrica_sim_baixa_tensao
0,Abrantes,2022,8,11797578,20260.0,293.0,4452.0,83.0,22194.0,2.0,...,20260,0,293,0,4318,134,83,0,22194,2
1,Abrantes,2022,9,11475993,20270.0,293.0,4445.0,81.0,22596.0,2.0,...,20270,0,293,0,4311,134,81,0,22596,2
2,Abrantes,2022,10,11789357,20280.0,293.0,4436.0,81.0,23126.0,2.0,...,20280,0,293,0,4302,134,81,0,23126,2
3,Abrantes,2022,11,11804430,20291.0,293.0,4442.0,80.0,23938.0,2.0,...,20291,0,293,0,4308,134,80,0,23938,2
4,Abrantes,2022,12,12050308,20297.0,294.0,4446.0,82.0,24420.0,2.0,...,20297,0,294,0,4312,134,82,0,24420,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,Óbidos,2023,9,5790012,8533.0,157.0,1731.0,107.0,14876.0,4.0,...,8533,0,157,0,1666,65,107,0,14876,4
4910,Óbidos,2023,10,5943510,8550.0,157.0,1730.0,107.0,15234.0,4.0,...,8550,0,157,0,1665,65,107,0,15234,4
4911,Óbidos,2023,11,5856102,8556.0,157.0,1725.0,110.0,15594.0,4.0,...,8556,0,157,0,1659,66,110,0,15594,4
4912,Óbidos,2023,12,6680626,8576.0,157.0,1726.0,107.0,16382.0,4.0,...,8576,0,157,0,1660,66,107,0,16382,4


9. Remover as colunas antigas

In [12]:
# Drop das colunas originais
merged_df.drop(columns=['energia_ativa_(kwh)', 'cpes_doméstico', 'cpes_iluminação_pública', 'cpes_não_doméstico', 'cpes_outros', 'cpes_mobilidade_eletrica_não', 'cpes_mobilidade_eletrica_sim'], inplace=True)

merged_df.info()

file_path = data_path + 'treated/merged-dataset-concelho.csv'

merged_df.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4914 entries, 0 to 4913
Data columns (total 15 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   concelho                                   4914 non-null   object
 1   ano                                        4914 non-null   int64 
 2   mês                                        4914 non-null   int64 
 3   energia_ativa_baixa_tensao_kwh             4914 non-null   int64 
 4   energia_ativa_alta_tensao_kwh              4914 non-null   int64 
 5   cpes_domestico_baixa_tensao                4914 non-null   int64 
 6   cpes_domestico_alta_tensao                 4914 non-null   int64 
 7   cpes_iluminacao_publica_baixa_tensao       4914 non-null   int64 
 8   cpes_iluminacao_publica_alta_tensao        4914 non-null   int64 
 9   cpes_nao_domestico_baixa_tensao            4914 non-null   int64 
 10  cpes_nao_domestico_alta_tensao      

10. Remover a coluna do concelho porque não pode fazer parte das variáveis informativas. Só serve para agregar os dados

In [13]:
merged_df.drop(columns=['concelho'], inplace=True)

11. Guardar ficheiro para verificar se está tudo certo

In [14]:
# Verificar nulls
null_counts = merged_df.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

file_path = data_path + 'treated/merged-dataset.csv'

merged_df.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

Número de nulls por coluna:
ano                                          0
mês                                          0
energia_ativa_baixa_tensao_kwh               0
energia_ativa_alta_tensao_kwh                0
cpes_domestico_baixa_tensao                  0
cpes_domestico_alta_tensao                   0
cpes_iluminacao_publica_baixa_tensao         0
cpes_iluminacao_publica_alta_tensao          0
cpes_nao_domestico_baixa_tensao              0
cpes_nao_domestico_alta_tensao               0
cpes_outros_baixa_tensao                     0
cpes_outros_alta_tensao                      0
cpes_mobilidade_eletrica_nao_baixa_tensao    0
cpes_mobilidade_eletrica_sim_baixa_tensao    0
dtype: int64
DataFrame guardado em  ../data/treated/merged-dataset.csv
